In [ ]:
%cd ../image_generation

In [ ]:
import os 

os.environ["HF_HUB_OFFLINE"] = "1"

In [ ]:
import torch 
import matplotlib.pyplot as plt 

device = torch.device("cuda:0")

In [ ]:
from diffusers import DDPMPipeline

# model_id = "google/ddpm-ema-celebahq-256"
model_id = os.path.abspath("./models/ddpm-ema-celebahq-256/")

ddpm_pipeline: DDPMPipeline = DDPMPipeline.from_pretrained(pretrained_model_name_or_path=model_id).to(device)

In [ ]:
image = ddpm_pipeline().images[0]
plt.imshow(image)

In [ ]:
# pipeline 的 __call__ 方法

import torch 
from torch import Tensor 
from numpy import ndarray
from diffusers.models.unet_2d import UNet2DModel
from diffusers.schedulers import DDPMScheduler

batch_size = 1
unet_model: UNet2DModel = ddpm_pipeline.unet
scheduler: DDPMScheduler = ddpm_pipeline.scheduler

image_shape = (
    batch_size, 
    ddpm_pipeline.unet.config.in_channels, 
    ddpm_pipeline.unet.config.sample_size,
    ddpm_pipeline.unet.config.sample_size,
)
image = torch.randn(image_shape, generator=None, dtype=None, layout=torch.strided).to(device)

# set step values
scheduler.set_timesteps(1000)

with torch.no_grad():
    for t in ddpm_pipeline.progress_bar(ddpm_pipeline.scheduler.timesteps):
        # 1. predict noise model_output
        model_output = unet_model.forward(image, t).sample

        # 2. compute previous image: x_t -> x_t-1
        image = scheduler.step(model_output, t, image).prev_sample

image = (image / 2 + 0.5).clamp(0, 1)  # image 从 [-1, 1] rescale 到 [0, 1]
image = image.cpu().permute(0, 2, 3, 1).numpy()  # 变成 ndarray 对象

plt.imshow(image[0])

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("imagefolder", name="anime", data_dir="./images/anime", drop_labels=True, keep_in_memory=True)

# dataset.save_to_disk("hf_anime")